Парсил tez-tour.
Не очень понял пункт 2 варианта 2:
Цитата:
2.Постройте распределение числа туров в определенной ценовой категории (определите пороги самостоятельно) для некоторой выборки связок страна-город.
Конец цитаты.

Прошелся по всем доступным на tez-tour странам с дефолтными настройками парсил все цены и считал средние для стран.

In [1]:
from selenium import webdriver 
from selenium.webdriver.support.ui import Select

import time

import pandas as pd
import numpy as np
import csv

import matplotlib.cm

import matplotlib.colors

import gmaps
import gmaps.datasets
import gmaps.geojson_geometries

In [ ]:
'''
Эти функции мы запихнем в map, чтобы убрать символы валют из всех элементов списка цен.
'''

def remove_eur(s):
    return s[:-4]

def remove_usd(s):
    return s[1:]

def remove_rub(s):
    return s[:-2]

In [ ]:
'''
Очищаем строки цен от символов валюты, считаем среднее, пересчитываем в рубли.
'''

def avg_price_by_country(xp_prices):
    usd = 59.38
    eur = 67.38
    try:
        if xp_prices[0][0] == '$':
            xp_prices = map(remove_usd, xp_prices)
            xp_prices = list(map(lambda x: x.replace(' ', ''), xp_prices))
            xp_prices = np.array(xp_prices).astype(np.float)
            xp_prices_rub_avg = np.mean(xp_prices) * usd

        elif xp_prices[0][-1] == 'R':
            xp_prices = map(remove_eur, xp_prices)
            xp_prices = list(map(lambda x: x.replace(' ', ''), xp_prices))
            xp_prices = np.array(xp_prices).astype(np.float)
            xp_prices_rub_avg = np.mean(xp_prices) * eur

        else:
            xp_prices = map(remove_rub, xp_prices)
            xp_prices = list(map(lambda x: x.replace(' ', ''), xp_prices))
            xp_prices = np.array(xp_prices).astype(np.float)
            xp_prices_rub_avg = np.mean(xp_prices)
            
    except:
        xp_prices_rub_avg = np.NaN
        
    return xp_prices_rub_avg

In [ ]:
'''
Парсим tez-tour. В результате получим два списка: страны и средние цены на тур в рублях.
'''
driver = webdriver.Chrome(executable_path='/home/vanya/double_data/chromedriver')

# В этих двух списках будем хранить страны и средние цены.
ls_countries = []
ls_avg_prices = []

url = 'https://www.tez-tour.com/search.html'

driver.get(url)

# tez-tour не любит, когда диапазон дат для поиска тура больше 18 дней.
driver.find_element_by_id('st-after').clear()
driver.find_element_by_id('st-after').send_keys('13.07.2017')
driver.find_element_by_id('st-before').clear()
driver.find_element_by_id('st-before').send_keys('31.07.2017')

# Делаем список опций для выпадающего списка стран.
country_select = driver.find_element_by_id('countries')
select = Select(country_select)
options = select.options

# Проходимся по списку опций стран.
for index in range(0, len(options) - 1):
    # По Египту и Израилю можно выбрать опции из списка, но цены по ним не показывают, но остаются предыдущие
    # (от Доминиканы).
    if options[index].text not in ['Египет', 'Израиль']:
        select.select_by_index(index)

        # Обходим поп-апы.
        try:      
            driver.find_element_by_id('fancybox-close').click()
            time.sleep(15)
        except:
            pass
        try:      
            driver.find_element_by_class_name('webim_close').click()
            time.sleep(15)
        except:
            pass
        time.sleep(20)
        # Нажимаем на кнопку найти.
        driver.find_element_by_class_name('gStatTourSearch').click()
        time.sleep(15)
        # Собираем цены по одной стране в список.
        el_prices = driver.find_elements_by_class_name('cardprice')
        prices = [x.text for x in el_prices]
        # Усредняем, записываем страну и цену в список.
        avg_price = avg_price_by_country(prices)
        country = options[index].text
        ls_avg_prices.append(avg_price)
        ls_countries.append(country)

        print(avg_price)
        print(country)
    else:
        continue
    
driver.quit()

In [ ]:
'''
Записываем в датафрейм списки стран и цен. Записываем датафрейм в файл.
'''
df = pd.DataFrame({'country': ls_countries, 'price': ls_avg_prices})
df.to_csv('country_data.csv')

In [2]:
'''
Читаем словарь для перевода названий стран на английский и данные с ценами, записанные в файл в ячейке сверху.
Перевод нужен, чтобы построить тепловую карту.
'''
with open("countries_rus.csv") as csv_codes:
    reader = csv.reader(csv_codes)
    codes_dict = {row[2]:row[1] for row in reader}
    
df_graph = pd.read_csv('country_data.csv')
df_graph["country_eng"] = df_graph["country"].map(codes_dict)

In [3]:
'''
Рейтинг
'''
df_rating = df_graph[['price', 'country']].dropna()

df_rating_sorted = df_rating.sort_values(['price', 'country'], ascending=[True,False])
df_rating_sorted.index = range(1, len(df_rating_sorted) + 1)

df_rating_sorted

,price,country
1,29236.8558,Греция
2,31959.3100,Беларусь
3,34513.6600,Россия
4,36089.3826,Турция
5,41193.4368,Латвия
6,41232.5172,Болгария
7,41813.3328,Чехия
8,41892.8412,Литва
9,46890.4158,Кипр
10,47063.5824,Австрия


In [4]:
'''
Строим тепловую карту.
'''
# Я не стал показывать свой API key.
gmaps.configure(api_key="AI")

countries_geojson = gmaps.geojson_geometries.load_geometry('countries')

min_price = min(df_graph['price'])
max_price = max(df_graph['price'])
price_range = max_price - min_price

def calculate_color(price):
    # Нормируем цену.
    normalized_price = (price - min_price) / price_range
    # Большая цена - темнее цвет страны.
    inverse_price = 1.0 - normalized_price
    # Делаем цену цветом.
    mpl_color = matplotlib.cm.gnuplot(inverse_price)
    gmaps_color = matplotlib.colors.rgb2hex(mpl_color)

    return gmaps_color

# Получаем цвет для каждой страны.
colors = []
for feature in countries_geojson['features']:
    country_name = feature['properties']['name']
    try:
        price = df_graph.loc[df_graph['country_eng'] == country_name, 'price'].iloc[0]
        color = calculate_color(price)
    except IndexError:
        color = (0, 0, 0, 0.3)
    colors.append(color)
    
fig = gmaps.figure()
price_layer = gmaps.geojson_layer(countries_geojson, fill_color=colors, stroke_color=colors, fill_opacity=0.8)
fig.add_layer(price_layer)
fig

In [ ]:
'''
Без API key карта не появится, поэтому приложил картинку
'''

![alt text](map.png "Карта")